In [1]:
import re
import matplotlib.image as mpimg
import numpy as np
from PIL import Image

In [2]:
PIXEL_DEPTH = 255
def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * PIXEL_DEPTH).round().astype(np.uint8)
    return rimg

In [3]:
def make_img_overlay(img, predicted_img):
    w = img.shape[0]
    h = img.shape[1]
    color_mask = np.zeros((w, h, 3), dtype=np.uint8)
    color_mask[:,:,0] = predicted_img*PIXEL_DEPTH

    img8 = img_float_to_uint8(img)
    background = Image.fromarray(img8, 'RGB').convert("RGBA")
    overlay = Image.fromarray(color_mask, 'RGB').convert("RGBA")
    new_img = Image.blend(background, overlay, 0.2)
    return new_img

In [4]:
# assign a label to a patch
def patch_to_label(patch):
    th = 0.25
    df = np.mean(patch)
    if df > th:
        return 1
    else:
        return 0

In [11]:
def summary_to_label(patch, n=2):
    th = 2.95
    df = np.mean(patch)
    #print(patch)
    #print(df)
    #print(np.amax(patch))
    if df >= th:
        return 1
    else:
        return 0

In [12]:
def mask_to_submission_strings(im, img_number):
    """Reads a single image and outputs the strings that should go into the submission file"""
    patch_size = 16
    for j in range(0, im.shape[1], patch_size):
        for i in range(0, im.shape[0], patch_size):
            patch = im[i:i + patch_size, j:j + patch_size]
            label = patch_to_label(patch)
            yield("{:03d}_{}_{},{}".format(img_number+1, j, i, label))

In [13]:
def masks_to_submission(submission_filename, *image_files):
    """Converts images into a submission file"""
    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')         
        for ind, fn in enumerate(image_files[0:]):
            f.writelines('{}\n'.format(s) for s in mask_to_submission_strings(fn, ind))

In [14]:
def collect_masks(name, ending, patches, id):
    TEST_IMG_SIZE = 608
    img = np.zeros((TEST_IMG_SIZE,TEST_IMG_SIZE))
    for p in patches:
        imagename = name + str(p) + '_' + str(id) + ending
        print(imagename)
        image = mpimg.imread(imagename)
        for j in range(0, image.shape[1], p):
            for i in range(0, image.shape[0], p):
                label = np.mean(image[i:i + p, j:j + p])
                #pred_label = np.mean(img[i:i + p, j:j + p])
                # set new label
                img[i:i + p, j:j + p] = img[i:i + p, j:j + p]+label
    
    p = 2
    for j in range(0, img.shape[1], p):
            for i in range(0, img.shape[0], p):
                patch = img[i:i + p, j:j + p]
                label = summary_to_label(patch)
                img[i:i + p, j:j + p] = label
            
    testname = 'test/test_'+ str(id) + '/test_' + str(id) + '.png'
    testimg = mpimg.imread(testname)        
    oimg = make_img_overlay(testimg, img)
    oimg.save('predictions_test/' + "overlay-comb_" + str(id) + ".png")
                
    return img
    

In [15]:
def save(submission_filename):
    #subm#ission_filename = 'submission_07.csv'
    patches = [2,8,16]
    images = []
    #for i in range(1, 51):
    for i in range(1, 51):
        images.append(collect_masks('predictions_test/prediction', '.png', patches, i))
    masks_to_submission(submission_filename, *images)


In [ ]:
save('submission_lukas_2-8-16.csv')

predictions_test/prediction2_1.png
predictions_test/prediction8_1.png
predictions_test/prediction16_1.png
predictions_test/prediction2_2.png
predictions_test/prediction8_2.png
predictions_test/prediction16_2.png
predictions_test/prediction2_3.png
predictions_test/prediction8_3.png
predictions_test/prediction16_3.png
predictions_test/prediction2_4.png
predictions_test/prediction8_4.png
predictions_test/prediction16_4.png
predictions_test/prediction2_5.png
predictions_test/prediction8_5.png
predictions_test/prediction16_5.png
predictions_test/prediction2_6.png
predictions_test/prediction8_6.png
predictions_test/prediction16_6.png
predictions_test/prediction2_7.png
predictions_test/prediction8_7.png
predictions_test/prediction16_7.png
predictions_test/prediction2_8.png
predictions_test/prediction8_8.png
predictions_test/prediction16_8.png
predictions_test/prediction2_9.png
predictions_test/prediction8_9.png
predictions_test/prediction16_9.png
predictions_test/prediction2_10.png
prediction